In [1]:
# 载入显示库
import ipywidgets.widgets as widgets
from IPython.display import display
import libcamera
import cv2
from picamera2 import Picamera2
import threading # 线程W
# 载入yolo图像识别库和语言播报库
from ultralytics import YOLO

In [2]:
def get_camera():
    # 获取第一个摄像头
    picamera_0 = Picamera2(camera_num=0)
    config = picamera_0.create_preview_configuration(main={"format": 'RGB888', "size": (180, 120)},
                                               raw={"format": "SRGGB12", "size": (720, 480)})
    config["transform"] = libcamera.Transform(hflip=1, vflip=1)
    picamera_0.configure(config)
    picamera_0.start()
    
    # 获取第二个摄像头
    picamera_1 = Picamera2(camera_num=1)
    config = picamera_1.create_preview_configuration(main={"format": 'YUYV', "size": (1280, 720)},
                                                   raw={"format": "YUYV", "size": (1280, 720)})
    config["transform"] = libcamera.Transform(hflip=0, vflip=1)
    picamera_1.configure(config)
    picamera_1.start()
    return picamera_0,picamera_1

def bgr8_to_jpeg(value, quality=75):
    return bytes(cv2.imencode('.jpg', value)[1])

def yolo_detect(picamera_1, frame_counter, model):
    # Load a pretrained YOLO11n model
    #model = YOLO("best_ncnn_model")
    frame_1 = picamera_1.capture_array()
    # YUV 转 BGR
    frame_bgr = cv2.cvtColor(frame_1, cv2.COLOR_YUV2BGR_YUYV) 
    if frame_counter >= 1:
        results = model.predict(frame_bgr)  # 只在这里用模型推理
        #print("识别结果为：", results)
        '''
        # 输出识别结果
        for result in results:
            boxes = result.boxes  # Boxes 对象
            names = result.names  # 类别字典

            if boxes is not None:
                for cls in boxes.cls:
                    print("识别种类：", names[int(cls)])
        '''
        # 绘制检测框
        annotated_frame = results[0].plot()  # 返回带框的图像
        # 显示或赋值给你原先的 widgets.Image
        image_1.value = bgr8_to_jpeg(annotated_frame)
        frame_counter = 0  # 计数器归零
    else:
        image_1.value = bgr8_to_jpeg(frame_bgr)
    return results

In [3]:
# 云台摄像头可视化
image_1 = widgets.Image(format='jpeg', width=180, height=120)
# image = widgets.Image(format='jpeg', width=1920, height=1080)
display(image_1)

Image(value=b'', format='jpeg', height='120', width='180')

In [4]:
# 初始化摄像头
# picamera_0是巡线云台摄像头，picamera_1是识别摄像头
picamera_0, picamera_1 = get_camera()
frame_counter = 1
# 读取模型
model = YOLO("../best_ncnn_model")
while True:
    yolo_detect(picamera_1, frame_counter, model)

WARNING ⚠️ Unable to automatically guess model task, assuming 'task=detect'. Explicitly define task for your model, i.e. 'task=detect', 'segment', 'classify','pose' or 'obb'.


[0:46:18.976643451] [3458]  INFO Camera camera_manager.cpp:325 libcamera v0.3.2+27-7330f29b
[0:46:19.009093226] [3488]  INFO RPI pisp.cpp:695 libpisp version v1.0.7 28196ed6edcf 29-08-2024 (16:33:32)
[0:46:19.024238422] [3488]  INFO RPI pisp.cpp:1154 Registered camera /base/axi/pcie@120000/rp1/i2c@80000/ov5647@36 to CFE device /dev/media0 and ISP device /dev/media1 using PiSP variant BCM2712_D0
[0:46:19.140690802] [3488]  WARN V4L2 v4l2_pixelformat.cpp:346 Unsupported V4L2 pixel format H264
[0:46:19.145192788] [3458]  INFO Camera camera.cpp:1197 configuring streams: (0) 180x120-RGB888 (1) 1920x1080-SGRBG16
[0:46:19.145316492] [3488]  INFO RPI pisp.cpp:1450 Sensor: /base/axi/pcie@120000/rp1/i2c@80000/ov5647@36 - Selected sensor format: 1920x1080-SGRBG10_1X10 - Selected CFE format: 1920x1080-GR16
[0:46:19.268084260] [3458]  INFO Camera camera.cpp:1197 configuring streams: (0) 1280x720-YUYV
[0:46:19.618417561] [3488]  INFO V4L2 v4l2_videodevice.cpp:1746 /dev/video0[148:cap]: Zero sequence

Loading ../best_ncnn_model for ncnn inference...

0: 640x640 1 仙人球, 130.7ms
Speed: 12.3ms preprocess, 130.7ms inference, 3.1ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 仙人球, 134.1ms
Speed: 6.7ms preprocess, 134.1ms inference, 2.1ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 仙人球, 151.5ms
Speed: 6.8ms preprocess, 151.5ms inference, 2.8ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 仙人球, 125.1ms
Speed: 9.0ms preprocess, 125.1ms inference, 1.7ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 仙人球, 122.7ms
Speed: 6.7ms preprocess, 122.7ms inference, 1.7ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 仙人球, 122.4ms
Speed: 8.9ms preprocess, 122.4ms inference, 2.6ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 仙人球, 128.3ms
Speed: 7.8ms preprocess, 128.3ms inference, 3.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 仙人球, 121.6ms
Speed: 7.5ms preprocess, 121.6ms inference, 2.7m

KeyboardInterrupt: 